**used libraries:**
- pandas
- glob
- os 
- pyunpack
- shutil

**make a new folder in data for preprocessing**

In [31]:
mkdir ../../data/preprocessing/


**download the data dump**

In [32]:
!wget  https://s3.amazonaws.com/wikia_xml_dumps/e/el/elderscrolls_pages_current.xml.7z -P ../../data/preprocessing/

--2023-04-14 08:32:51--  https://s3.amazonaws.com/wikia_xml_dumps/e/el/elderscrolls_pages_current.xml.7z
Auflösen des Hostnamens s3.amazonaws.com (s3.amazonaws.com)… 54.231.233.160, 52.217.116.136, 52.216.52.240, ...
Verbindungsaufbau zu s3.amazonaws.com (s3.amazonaws.com)|54.231.233.160|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 191673320 (183M) [application/x-7z-compressed]
Wird in »../../data/preprocessing/elderscrolls_pages_current.xml.7z« gespeichert.

elderscrolls_pages_ 100%[===================>] 182,79M  29,3MB/s    in 7,4s    

2023-04-14 08:32:59 (24,8 MB/s) - »../../data/preprocessing/elderscrolls_pages_current.xml.7z« gespeichert [191673320/191673320]



**clone and unzip wikiextractor**

In [33]:
!wget https://github.com/attardi/wikiextractor/archive/master.zip -P ../../data/preprocessing/
!unzip ../../data/preprocessing/master.zip -d ../../data/preprocessing/

--2023-04-14 08:32:59--  https://github.com/attardi/wikiextractor/archive/master.zip
Auflösen des Hostnamens github.com (github.com)… 140.82.121.3
Verbindungsaufbau zu github.com (github.com)|140.82.121.3|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 302 Found
Platz: https://codeload.github.com/attardi/wikiextractor/zip/refs/heads/master [folgend]
--2023-04-14 08:32:59--  https://codeload.github.com/attardi/wikiextractor/zip/refs/heads/master
Auflösen des Hostnamens codeload.github.com (codeload.github.com)… 140.82.121.10
Verbindungsaufbau zu codeload.github.com (codeload.github.com)|140.82.121.10|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: nicht spezifiziert [application/zip]
Wird in »../../data/preprocessing/master.zip« gespeichert.

master.zip              [ <=>                ]  48,29K  --.-KB/s    in 0,03s   

2023-04-14 08:32:59 (1,62 MB/s) - »../../data/preprocessing/master.zip« gespeichert [49444]

Archive:  

**unpack the data dump**

In [34]:
from pyunpack import Archive

Archive('../../data/preprocessing/elderscrolls_pages_current.xml.7z').extractall("../../data/preprocessing/")

**use wikiextractor to clean the data**
- cleaned data will be saved in json in `../../data/preprocessing/text`

In [35]:
!mkdir ../../data/preprocessing/text
!python3 -m wikiextractor.WikiExtractor --json -o ../../data/preprocessing/text ../../data/preprocessing/elderscrolls_pages_current.xml

INFO: Preprocessing '../../data/preprocessing/elderscrolls_pages_current.xml' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Loaded 2444 templates in 15.7s
INFO: Starting page extraction from ../../data/preprocessing/elderscrolls_pages_current.xml.
INFO: Using 7 extract processes.
INFO: Finished 7-process extraction of 82931 articles in 38.6s (2149.0 art/s)


**create one dataframe from all data files**

In [36]:
import glob
import os
import pandas as pd

df = pd.DataFrame()

# default output directory is ../../data/preprocessing/text 

for x in os.walk("../../data/preprocessing/text"):
    for y in glob.glob(os.path.join(x[0], '**')):
        if not os.path.isdir(y):
            df = pd.concat([df, pd.read_json(y, lines=True)], ignore_index=True, sort=False)       


df


,id,revid,url,title,text
0,994497,32045548,http://elderscrolls.fandom.com/wiki?curid=994497,The Imperfect Logistics Log,The Imperfect Logistics Log is a book in .\nCo...
1,994500,32045548,http://elderscrolls.fandom.com/wiki?curid=994500,The Law of Gears,The Law of Gears is a book in .\nContents.\nBy...
2,994501,32045548,http://elderscrolls.fandom.com/wiki?curid=994501,The Memories of Sotha Sil,The Memories of Sotha Sil is a book in .\nCont...
3,994504,32045548,http://elderscrolls.fandom.com/wiki?curid=994504,The Mystery of Artaeum,The Mystery of Artaeum is a book in .\nContent...
4,994505,32045548,http://elderscrolls.fandom.com/wiki?curid=994505,The Mystery of Factotums,The Mystery of Factotums is a book in .\nConte...
...,...,...,...,...,...
82925,954666,32045548,http://elderscrolls.fandom.com/wiki?curid=954666,Nothilgor,Nothilgor is a Bosmer member of the Aldmeri Do...
82926,954667,17401442,http://elderscrolls.fandom.com/wiki?curid=954667,Escaped Prisoner,Escaped Prisoner is an unused Nord in . He wea...
82927,954669,32045548,http://elderscrolls.fandom.com/wiki?curid=954669,Nothirvir,Nothirvir is a Bosmer member of the Vinedusk R...
82928,954670,32045548,http://elderscrolls.fandom.com/wiki?curid=954670,Nothrelras,Nothrelras is a Bosmer soldier of the Aldmeri ...


**delete unnecessary data and save dataframe as .pickle file**
- dataframe can be read with `pd.read_pickle('../../data/preprocessing/preprocessed_data.pickle')` 

In [25]:
import shutil
shutil.rmtree('../../data/preprocessing')

In [ ]:
!mkdir ../../data/preprocessing
df.to_pickle('../../data/preprocessing/preprocessed_data.pickle')